In [174]:
import networkx as nx
from networkx.algorithms import bipartite
import numpy as np
import math
import random
import matplotlib.pyplot as plt
import xlwt
import csv
import itertools
from itertools import izip_longest
import collections
from collections import defaultdict
import psycopg2 as pg
import time
#import powerlaw
import pandas as pd
import operator

In [95]:
df = pd.read_csv('wordproportionsOrdered30.csv')

In [96]:
MPfiles = df['Files'].values.tolist()

In [97]:
print len(MPfiles)

2097


In [98]:
print MPfiles[0]

file:/home/dvasques/softwares/mallet/hansard/speech-data-2/adams-amy/49-2009.txt


In [99]:
#function to find positioning of forward bar (to get only the mp name)
def dash_positions(url):
    
    s = url
    c = '/'
    positions = [pos for pos, char in enumerate(s) if char == c]
    #print speech_urls[205][0].index('-')
    
    return positions

In [100]:
def get_mp(MPfiles):
    
    mps = []
    
    for i in range(len(MPfiles)):
        
        positions = dash_positions(MPfiles[i])

        mp_name = MPfiles[i][positions[6]+1:positions[7]] #string
        mps.append(mp_name)
        
    return mps

In [193]:
def get_years(MPfiles):
    
    years = []
    
    for i in range (len(MPfiles)):
        
        positions = dash_positions(MPfiles[i])
        
        year = MPfiles[i][positions[7]+4:positions[7]+8] #string
        years.append(year)
        
    return years

In [214]:
def get_terms(MPfiles):
    
    terms = []
    
    for i in range (len(MPfiles)):
        
        positions = dash_positions(MPfiles[i])
        
        term = MPfiles[i][positions[7]+1:positions[7]+3] #string
        terms.append(term)
        
    return terms

In [216]:
mps = get_mp(MPfiles)
print len(mps)

years = get_years(MPfiles)
print len(years)

terms = get_terms(MPfiles)
print len(terms)

2097
2097
2097


In [102]:
topic0 = df['Topic0'].values.tolist()
topic1 = df['Topic1'].values.tolist()
topic2 = df['Topic2'].values.tolist()
topic3 = df['Topic3'].values.tolist()
topic4 = df['Topic4'].values.tolist()
topic5 = df['Topic5'].values.tolist()
topic6 = df['Topic6'].values.tolist()
topic7 = df['Topic7'].values.tolist()
topic8 = df['Topic8'].values.tolist()
topic9 = df['Topic9'].values.tolist()
topic10 = df['Topic10'].values.tolist()
topic11 = df['Topic11'].values.tolist()
topic12 = df['Topic12'].values.tolist()
topic13 = df['Topic13'].values.tolist()
topic14 = df['Topic14'].values.tolist()
topic15 = df['Topic15'].values.tolist()
topic16 = df['Topic16'].values.tolist()
topic17 = df['Topic17'].values.tolist()
topic18 = df['Topic18'].values.tolist()
topic19 = df['Topic19'].values.tolist()
topic20 = df['Topic20'].values.tolist()
topic21 = df['Topic21'].values.tolist()
topic22 = df['Topic22'].values.tolist()
topic23 = df['Topic23'].values.tolist()
topic24 = df['Topic24'].values.tolist()
topic25 = df['Topic25'].values.tolist()
topic26 = df['Topic26'].values.tolist()
topic27 = df['Topic27'].values.tolist()
topic28 = df['Topic28'].values.tolist()
topic29 = df['Topic29'].values.tolist()

In [103]:
print type(topic0[0])

<type 'float'>


In [218]:
df2 = pd.read_csv('countTerms.csv')

In [219]:
mp_unique = df2['Name'].values.tolist()
party = df2['Party'].values.tolist()
presence = df2['Presence'].values.tolist()

In [220]:
mp_party = []
mp_presence = []

for mp in mps:
    for i in range(len(mp_unique)):
        if mp == mp_unique[i]:
            mp_party.append(party[i])
            mp_presence.append(presence[i])
            #print mp, mp_unique[i], party[i]
            

In [221]:
print len(mp_party)
print len(mp_presence)

2097
2097


In [108]:
for mp in mps:
    if mp not in mp_unique:
        print mp

In [118]:
for i in range(0,10,1):
    print mps[i], mp_party[i]

adams-amy National
adams-amy National
adams-amy National
adams-amy National
adams-amy National
adams-amy National
adams-amy National
adams-amy National
adams-amy National
adams-amy National


In [139]:
topic_list = []

for i in range(len(mps)):
    list_mp = []
    t = 0
    for topic in [topic0, topic1, topic2, topic3, topic4, topic5, topic6, topic7, topic8, topic9, topic10, topic11, topic12, topic13, topic14, topic15, topic16, topic17, topic18, topic19, topic20, topic21, topic22, topic23, topic24, topic25, topic26, topic27, topic28, topic29]:
        if topic[i] > 0.15:
            list_mp.append(str(t))
        t += 1
    topic_list.append(list_mp)
            

In [140]:
print len(topic_list)

2097


In [141]:
for i in range(10):
    print topic_list[i]

['7']
['2', '27', '29']
['6', '27', '29']
['14', '29']
['8', '25', '29']
['19']
['7', '29']
['14', '27', '29']
['2']
['25', '29']


In [223]:
#create network with four war types, with type of war as attribute
def create_network(mps, topic_list, mp_party, terms, years, term=None, year=None):

    t0 = time.time()
    
    #CREATE NETWORK
    B = nx.Graph()
    #for i in range(len(war_extra)):
        #B.add_node(war_extra[i], bipartite=0, kind='extra_state')
        #B.add_node(state_extra[i], bipartite=1)
        #B.add_edge(war_extra[i],state_extra[i])
    
    if year == None and term == None:
        for i in range(len(mps)):
            B.add_node(mps[i], bipartite=1, party=mp_party[i], presence=mp_presence[i])
            for topic in topic_list[i]:
                B.add_node(topic, bipartite=0)
                B.add_edge(mps[i],topic)
                
    else:
        if year == None:
            for i in range(len(mps)):
                if terms[i] == term:
                    B.add_node(mps[i], bipartite=1, party=mp_party[i], presence=mp_presence[i])
                    for topic in topic_list[i]:
                        B.add_node(topic, bipartite=0)
                        B.add_edge(mps[i],topic)
                        
        else:
            for i in range(len(mps)):
                if years[i] == year:
                    B.add_node(mps[i], bipartite=1, party=mp_party[i], presence=mp_presence[i])
                    for topic in topic_list[i]:
                        B.add_node(topic, bipartite=0)
                        B.add_edge(mps[i],topic)           
            
        
    #for i in range(len(war_intra)):
        #B.add_node(war_intra[i], bipartite=0, kind='intra_state')
        #B.add_node(state_intra[i], bipartite=1)
        #B.add_edge(war_intra[i],state_intra[i])
        
    #for i in range(len(war_non)):
        #B.add_node(war_non[i], bipartite=0, kind='non_state')
        #B.add_node(entity_non[i], bipartite=1)
        #B.add_edge(war_non[i],entity_non[i])

    print 'Running time to create network: ' + str(time.time() - t0)
    
    return B

In [143]:
B = create_network(mps, topic_list, mp_party, years)

Running time to create network: 0.0407841205597


In [144]:
for topic in topic_list[0]:
    print topic

7


In [145]:
def create_bottom_projection(B):

    t0 = time.time()
    
    #BIPARTITE NETWORK
    top_nodes = set(node for node,d in B.nodes(data=True) if d['bipartite']==0) #dlist
    bottom_nodes = set(B) - top_nodes #klist
    #deg_top, deg_bottom = bipartite.degrees(B,bottom_nodes) #dictionary
    
    G_bottom = bipartite.projected_graph(B,bottom_nodes)

    print 'Running time to create projection: ' + str(time.time() - t0)
    
    return G_bottom

In [146]:
def create_bottom_weighted_projection(B):

    t0 = time.time()
    
    #BIPARTITE NETWORK
    top_nodes = set(node for node,d in B.nodes(data=True) if d['bipartite']==0) #dlist
    bottom_nodes = set(B) - top_nodes #klist
    #deg_top, deg_bottom = bipartite.degrees(B,bottom_nodes) #dictionary
    
    G_bottom = bipartite.weighted_projected_graph(B,bottom_nodes)

    print 'Running time to create projection: ' + str(time.time() - t0)
    
    return G_bottom

In [147]:
G_bottom = create_bottom_projection(B)

Running time to create projection: 0.13621711731


In [148]:
G_bottom_w = create_bottom_weighted_projection(B)

Running time to create projection: 0.163527011871


In [149]:
nx.write_graphml(G_bottom_w, "mps_topics_projection_w_threshold15.graphml")

In [150]:
top_nodes = set(node for node,d in B.nodes(data=True) if d['bipartite']==0) #dlist
bottom_nodes = set(B) - top_nodes #klist
deg_top, deg_bottom = bipartite.degrees(B,bottom_nodes) #dictionary

In [158]:
av_top_degree = sum(deg_top.values())/float(len(deg_top))
print av_top_degree

37.3


In [160]:
def bipartite_distribution(degree_dict):
    
    t0 = time.time()
    
    #Degree Distribution
    degree_list = degree_dict.values()
    P = []
    for d in range(0,max(degree_list)+1,1):
        if d in degree_list:
            f = degree_list.count(d)
            P.append(f)
        else:
            P.append(0)
            
    PDF = []
    CCDF = []
    for i, d in enumerate(P):
        PDF.append(float(d)/sum(P))
        CCDF.append(sum(P[i:])/float(sum(P)))
        
    print 'Running time to create distribution: ' + str(time.time() - t0)
        
    return CCDF, PDF

In [161]:
def projection_distribution(G):
    
    t0 = time.time()
    
    #Projection distribution
    degree_hist = nx.degree_histogram(G)
    q_list = []
    Pq = []
    for degree, num_of_nodes in enumerate(degree_hist):
        q_list.append(degree)
        Pq.append(num_of_nodes)
        
    PDF = []
    CCDF = []
    for i,q in enumerate(Pq):
        PDF.append(float(q)/sum(Pq))
        CCDF.append(sum(Pq[i:])/float(sum(Pq)))
        
    print 'Running time to create distribution: ' + str(time.time() - t0)  
    
    return CCDF, PDF

In [184]:
#calling PDF and CCDF from bipartite set
Pd_CCDF, Pd_PDF = bipartite_distribution(deg_top)
Pk_CCDF, Pk_PDF = bipartite_distribution(deg_bottom)
#calling PDF and CCDF from projections (simple)
Pq_CCDF, Pq_PDF = projection_distribution(G_bottom)

Running time to create distribution: 0.000756978988647
Running time to create distribution: 0.000154972076416
Running time to create distribution: 0.00252795219421


In [163]:
print Pd_PDF

[0.0, 0.0, 0.0, 0.0, 0.0, 0.03333333333333333, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.03333333333333333, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1, 0.03333333333333333, 0.06666666666666667, 0.06666666666666667, 0.0, 0.03333333333333333, 0.0, 0.0, 0.03333333333333333, 0.06666666666666667, 0.0, 0.0, 0.03333333333333333, 0.03333333333333333, 0.0, 0.03333333333333333, 0.03333333333333333, 0.03333333333333333, 0.03333333333333333, 0.03333333333333333, 0.03333333333333333, 0.0, 0.03333333333333333, 0.0, 0.0, 0.0, 0.03333333333333333, 0.0, 0.0, 0.03333333333333333, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.03333333333333333, 0.0, 0.03333333333333333, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.03333333333333333, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.03333333333333333, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.03333333333333333]


In [164]:
print len(Pd_PDF)

120


In [186]:
Degree_d = list(range(len(Pd_PDF)))
Degree_q = list(range(len(Pq_PDF)))

In [189]:
plt.plot(Degree_q, Pq_CCDF,'ro')
plt.ylabel('CCDF')
plt.xlabel('Degree')
plt.show()

In [175]:
sorted_deg_top = sorted(deg_top.items(), key=operator.itemgetter(1))
print sorted_deg_top

[('4', 5), ('11', 13), ('24', 19), ('22', 19), ('12', 19), ('25', 20), ('1', 21), ('3', 21), ('21', 22), ('18', 22), ('17', 24), ('14', 27), ('20', 28), ('13', 28), ('28', 31), ('6', 32), ('26', 34), ('2', 35), ('15', 36), ('16', 37), ('23', 38), ('10', 39), ('7', 41), ('9', 45), ('0', 48), ('5', 58), ('8', 60), ('29', 82), ('27', 96), ('19', 119)]


In [190]:
plt.hist(deg_top.values(),bins=10)
plt.show()

In [211]:
B2003 = create_network(mps, topic_list, mp_party, years, '2003')

Running time to create network: 0.00133800506592


In [212]:
G_bottom_w2003 = create_bottom_weighted_projection(B2003)

Running time to create projection: 0.00944399833679


In [213]:
nx.write_graphml(G_bottom_w2003, "mps_topics_projection_w_threshold15_2003.graphml")

In [226]:
B48 = create_network(mps, topic_list, mp_party, terms, years, '48', year=None)
G_bottom_w48 = create_bottom_weighted_projection(B48)
nx.write_graphml(G_bottom_w48, "mps_topics_projection_w_threshold15_48.graphml")

top_nodes = set(node for node,d in B48.nodes(data=True) if d['bipartite']==0) #dlist
bottom_nodes = set(B48) - top_nodes #klist
deg_top, deg_bottom = bipartite.degrees(B48,bottom_nodes) #dictionary

Running time to create network: 0.0181930065155
Running time to create projection: 0.0321979522705


In [228]:
av_deg_top = sum(deg_top.values())/float(len(deg_top))
print av_deg_top

av_deg_bottom = sum(deg_bottom.values())/float(len(deg_bottom))
print av_deg_bottom

12.8965517241
2.94488188976


In [229]:
sorted_deg_top = sorted(deg_top.items(), key=operator.itemgetter(1))
print sorted_deg_top

[('25', 1), ('11', 1), ('14', 1), ('2', 3), ('10', 3), ('1', 4), ('24', 5), ('22', 5), ('0', 5), ('4', 5), ('26', 8), ('3', 8), ('18', 8), ('29', 9), ('12', 9), ('21', 11), ('17', 11), ('23', 12), ('13', 12), ('28', 14), ('6', 14), ('7', 16), ('15', 16), ('16', 16), ('5', 18), ('8', 34), ('9', 35), ('27', 44), ('19', 46)]
